# **Binary Classification with PyTorch**

In [17]:
import os
import sys 
from collections import Counter

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import PIL
import sklearn
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import torchinfo

ModuleNotFoundError: No module named 'torchinfo'